<a href="https://colab.research.google.com/github/Mahabubjamil/ColabNotebook/blob/main/201_15_13819.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [99]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn.functional as F
import os
import pandas as pd
from PIL import Image
from torch.utils.data import Dataset, DataLoader

In [100]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [101]:
num_classes = 10
num_epochs = 3
batch_size = 64
learning_rate = 0.001

In [102]:
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
])

In [103]:
trainset = torchvision.datasets.KMNIST(root='./data',train=True,
                                       download=True, transform=transform)

In [104]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                         shuffle=True)

In [105]:
len(trainset)

60000

In [106]:
len(trainloader)

938

In [107]:
testset = torchvision.datasets.KMNIST(root='./data',train=False,
                                       download=True, transform=transform)

In [108]:
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False)

In [109]:
for X, y in trainloader:
  print(X.shape)
  break

torch.Size([64, 1, 32, 32])


In [110]:
len(testloader)

157

In [111]:
len(testset)

10000

In [112]:
class ConvNet(nn.Module):

    def __init__(self):
        super(ConvNet,self).__init__()
        self.conv1= nn.Conv2d(3,6,5)
        self.bn1=nn.BatchNorm2d(6)
        self.conv2= nn.Conv2d(6,16,5)
        self.bn2=nn.BatchNorm2d(16)
        self.conv3= nn.Conv2d(3,6,5)
        self.bn1=nn.BatchNorm2d(6)
        self.conv4= nn.Conv2d(6,28,5)
        self.bn2=nn.BatchNorm2d(28)

        self.fc1= nn.Linear(28*5*5, 120) # 5*5 from image dimension
        self.bn3=nn.BatchNorm1d(120)
        self.fc2= nn.Linear(120,84)
        self.bn4=nn.BatchNorm1d(84)
        self.fc3= nn.Linear(84,10)

    def forward(self, input):
        x=self.conv1(input)
        x=F.relu(x)
        x=F.max_pool2d(x,(2,2))
        x=self.bn1(x)
        x=self.conv2(x)
        x=F.relu(x)
        x=F.max_pool2d(x,(2,2))
        x=self.bn2(x)

        x= torch.flatten(x,1)
        x=self.fc1(x)
        x=F.relu(x)
        x=self.bn3(x)
        x=self.fc2(x)
        x=F.relu(x)
        x=self.bn4(x)
        x=self.fc3(x)
        return x


In [113]:
model= ConvNet()
print(model)

ConvNet(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (bn1): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (bn2): BatchNorm2d(28, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv4): Conv2d(6, 28, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=700, out_features=120, bias=True)
  (bn3): BatchNorm1d(120, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (bn4): BatchNorm1d(84, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [114]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=.001)

In [115]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.trainloader) #total number of image
    model.train()

    for batch, (image, label) in enumerate(dataloader):

        pred = model(image)
        loss = loss_fn(pred, label)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(image)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [116]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    total_loss, correct = 0, 0

    with torch.no_grad():
        for image, label in dataloader:
            pred = model(image)
            total_loss += loss_fn(pred, label).item()
            correct += (pred.argmax(1) == label).type(torch.float).sum().item()

    total_loss /= num_batches
    correct /= size
    print(f"Test result: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {total_loss:>8f} \n")

In [117]:
true_labels = [] #for confusion matrix
predicted_labels = [] #for confusion matrix

